In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [2]:
url_template = "https://thuvienphapluat.vn/page/tim-van-ban.aspx?keyword=m%C3%A3+s%E1%BB%91&area=0&match=True&type=21&status=0&signer=0&sort=1&lan=1&scan=0&org=0&fields=4&page={}"


def get_links_from_page(page_number):
    url = url_template.format(page_number)
    response = requests.get(url)
    

    if response.status_code != 200:
        print(f"Lỗi khi tải trang {page_number}")
        return []


    soup = BeautifulSoup(response.text, 'html.parser')


    divs = soup.find_all('div', class_=re.compile(r'content-[01]'))


    links = [div.find('a')['href'] for div in divs if div.find('a')]
    
    return links


all_links = []
for page in range(1, 401):
    print(f"Đang tải trang {page}")
    links = get_links_from_page(page)
    all_links.extend(links)

print(f"Tổng số link thu thập được: {len(all_links)}")


Đang tải trang 1
Đang tải trang 2
Đang tải trang 3
Đang tải trang 4
Đang tải trang 5
Đang tải trang 6
Đang tải trang 7
Đang tải trang 8
Đang tải trang 9
Đang tải trang 10
Đang tải trang 11
Đang tải trang 12
Đang tải trang 13
Đang tải trang 14
Đang tải trang 15
Đang tải trang 16
Đang tải trang 17
Đang tải trang 18
Đang tải trang 19
Đang tải trang 20
Đang tải trang 21
Đang tải trang 22
Đang tải trang 23
Đang tải trang 24
Đang tải trang 25
Đang tải trang 26
Đang tải trang 27
Đang tải trang 28
Đang tải trang 29
Đang tải trang 30
Đang tải trang 31
Đang tải trang 32
Đang tải trang 33
Đang tải trang 34
Đang tải trang 35
Đang tải trang 36
Đang tải trang 37
Đang tải trang 38
Đang tải trang 39
Đang tải trang 40
Đang tải trang 41
Đang tải trang 42
Đang tải trang 43
Đang tải trang 44
Đang tải trang 45
Đang tải trang 46
Đang tải trang 47
Đang tải trang 48
Đang tải trang 49
Đang tải trang 50
Đang tải trang 51
Đang tải trang 52
Đang tải trang 53
Đang tải trang 54
Đang tải trang 55
Đang tải trang 56
Đ

In [3]:
with open('links.txt', 'w', encoding='utf-8') as f:
    for link in all_links:
        f.write(link + '\n')

print("Ghi thành công các link vào file 'links.txt'")

Ghi thành công các link vào file 'links.txt'
